## Jpype

In [ ]:
%load_ext autoreload
%autoreload 2
import nima_io
import nima_io.read as ir

# import skimage.io
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# nd2 = "/home/dan/pims/pims/tests/data/bioformats/cluster.nd2"
import numpy as np
from pathlib import Path

tdata = Path("../../tests/data/")

lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
# imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

# mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
# bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

slif = str(lif)
simg_tile = str(img_tile)
simg_void_tile = str(img_void_tile)
# simgsingle = str(imgsingle)
# smcts = str(mcts)
# sbigtiff = str(bigtiff)

Core metadata list at:
https://docs.openmicroscopy.org/bio-formats/5.9.1/developers/file-reader.html



In [ ]:
from six.moves.urllib.request import urlopen
import hashlib, os

url = (
    "http://downloads.openmicroscopy.org/bio-formats/"
    + "5.9.0"
    + "/artifacts/loci_tools.jar"
)
loc = "."
path = os.path.join(loc, "loci_tools.jar")
loci_tools = urlopen(url).read()
sha1_checksum = urlopen(url + ".sha1").read().split(b" ")[0].decode()

downloaded = hashlib.sha1(loci_tools).hexdigest()
if downloaded != sha1_checksum:
    raise IOError(
        "Downloaded loci_tools.jar has invalid checksum. " "Please try again."
    )

with open(path, "wb") as output:
    output.write(loci_tools)

In [ ]:
import jpype

# loci_path = '/home/dan/.venvs/imgread-dev/lib/python3.6/site-packages/pims/loci_tools.jar'
# loci_path = '/home/dan/.progs/bftools/bioformats_package.jar'
loci_path = "./loci_tools.jar"
java_memory = "512m"

jpype.startJVM(
    jpype.getDefaultJVMPath(),
    "-ea",
    "-Djava.class.path=" + loci_path,
    "-Xmx" + java_memory,
)

In [ ]:
filepath = simg_void_tile
loci = jpype.JPackage("loci")
rdr = loci.formats.ImageReader()
rdr.setMetadataStore(loci.formats.MetadataTools.createOMEXMLMetadata())
rdr.setId(filepath)
xml_md = rdr.getMetadataStore()

In [ ]:
md, mdd = ir.get_md_dict(xml_md, debug=True)
md

In [ ]:
print(md["ImageCount"][0][1], md["PlaneCount"])
print(md["PlanePositionX"][3][1][0])
print(md["ImageAcquisitionDate"])
print(md["PixelsPhysicalSizeX"][0][1][0])
print(md["PixelsType"][0][1])

print(md["PlanePositionZ"])

In [ ]:
md0, rdrdt = ir.read_jpype(simg_tile)
rdrdt, md0

In [ ]:
mmd = rdrdt[2]

In [ ]:
mmd["ArcType"]

In [ ]:
rdr = rdrdt[0]

In [ ]:
rdr.getPixelType(), rdr.getDimensionOrder(),

In [ ]:
r = rdr.getMetadataStoreRoot()

In [ ]:
reader = rdr.getReader()

In [ ]:
r.getImage

In [ ]:
rdr.getIndex(0, 3, 2)

In [ ]:
# loci = jpype.JPackage("loci")

isLittleEndian = rdr.isLittleEndian()
LE_prefix = [">", "<"][isLittleEndian]
FormatTools = loci.formats.FormatTools
_dtype_dict = {
    FormatTools.INT8: "i1",
    FormatTools.UINT8: "u1",
    FormatTools.INT16: LE_prefix + "i2",
    FormatTools.UINT16: LE_prefix + "u2",
    FormatTools.INT32: LE_prefix + "i4",
    FormatTools.UINT32: LE_prefix + "u4",
    FormatTools.FLOAT: LE_prefix + "f4",
    FormatTools.DOUBLE: LE_prefix + "f8",
}
_dtype_dict_java = {}
for loci_format in _dtype_dict.keys():
    _dtype_dict_java[loci_format] = (
        FormatTools.getBytesPerPixel(loci_format),
        FormatTools.isFloatingPoint(loci_format),
        isLittleEndian,
    )
# above duplicated

In [ ]:
def _jbytearr_javacasting(arr, dtype, bpp, fp, little_endian):
    # let java do the type conversion
    Jconv = loci.common.DataTools.makeDataArray(arr, bpp, fp, little_endian)
    return np.array(Jconv[:], dtype=dtype)


dtype = float

java_dtype = _dtype_dict_java[rdr.getPixelType()]
a_jbytearr_javacasting = lambda arr: _jbytearr_javacasting(arr, dtype, *java_dtype)
a_jbytearr_javacasting([1, 2])

In [ ]:
a_jbytearr_javacasting(rdr.openBytes(0)).shape

In [ ]:
cs = loci.formats.ChannelSeparator(reader)
fs = loci.formats.Memoizer(reader)

In [ ]:
v = xml_md.getPixelsPhysicalSizeY(0)
v.value(), v.unit, ir.convert_value(v)

In [ ]:
rdr.getMetadataStore()

In [ ]:
[(k, v) for k, v in mdd.items() if k[:9] == "getPixels"]

In [ ]:
md_, mdd_ = ir.get_md_dict(xml_md, simg_tile)
md

In [ ]:
for k in md_:
    if md_[k] != md[k]:
        print(md_[k], md[k])